# This Notebook is for Notetaking

## Creating Videos from the MEEP Package

Take a look at the code block below. They run the simulation into steady state. Note that in a simulation, you can always "pick up where you left off" by calling the run command again!

```python
sim.reset_meep()
fcen=0.118
df = 0.1
sim.sources = [mp.Source(mp.GaussianSource(fcen, fwidth=df), mp.Ez, mp.Vector3(r+0.1))]

# Start the simulation and get into steady state
sim.run(until=600) 

# Prepare the animator and record the steady state response
f = plt.figure(dpi=150)
Animate = mp.Animate2D(sim, fields=mp.Ez, f=f, realtime=False, normalize=True)
sim.run(mp.at_every(0.5,Animate),until=25)

# Close the animator's working frame
plt.close()

# Process the animation and view it
filename = "media/ring_simple.mp4"
Animate.to_mp4(5,filename)
Video(filename)
```

They can also create an Animate2D instance (there's also an Animate3D class). You need to have an active matplotlib figure while the simulation is running, and then you can create an mp4 video at the end. Of course if you want the png files instead, that may require that you actually save the fields instead. But just note that if you're not making a gif, most of the time you don't care about the fields during the simulation besides for the purpose of making a gif. 

## Material Dispersion

For a wavelength-dependent, purely-real permittivity (i.e., with no loss) which can be represented via the Sellmeier equation:

$$
\epsilon(\lambda) = 1 + \sum_{n} \frac{B_n \lambda^2}{\lambda^2 - C_n}
$$

where $\lambda$ is the vacuum wavelength, each term containing two coefficients ($B_n$ and $C_n$) can be directly transferred to a Lorentzian polarization field using a simple substitution of variables: $\omega_n = 1 \big/ \sqrt{C_n}$, $\gamma_n = 0$ and $\sigma_n = B_n$. Several examples of importing Sellmeier coefficients from published fitting data including [germanium](https://github.com/NanoComp/meep/blob/master/python/materials.py#L884-L901) (Ge) and [gallium nitride](https://github.com/NanoComp/meep/blob/master/python/materials.py#L1162-L1188) (GaN) are provided in the [Materials Library](https://meep.readthedocs.io/en/latest/Materials/#materials-library).

As an example, we can look at Lithium Niobate in MEEP's Materials library. 

They define $\gamma, \omega$ and $\sigma$ for several Lorentzian terms for the ordinary and extraordinary axis, and then generate the susceptibility by including each Lorentzian in a list. For defining new materials you can follow this template.

```python
# default unit length is 1 μm
um_scale = 1.0

# conversion factor for eV to 1/μm [=1/hc]
eV_um_scale = um_scale/1.23984193

# lithium niobate (LiNbO3) from D.E. Zelmon et al., J. Optical Society of America B, Vol. 14, pp. 3319-22 (1997)
# ref: https://refractiveindex.info/?shelf=main&book=LiNbO3&page=Zelmon-o
# ref: https://refractiveindex.info/?shelf=main&book=LiNbO3&page=Zelmon-e
# wavelength range: 0.4 - 5.0 μm

## NOTE: ordinary (o) axes in X and Y, extraordinary (e) axis in Z

LiNbO3_range = mp.FreqRange(min=um_scale/5.0, max=um_scale/0.4)

LiNbO3_frq1 = 1/(0.13281566172707193*um_scale)
LiNbO3_gam1 = 0
LiNbO3_sig1 = 2.6734
LiNbO3_frq2 = 1/(0.24318717071424636*um_scale)
LiNbO3_gam2 = 0
LiNbO3_sig2 = 1.2290
LiNbO3_frq3 = 1/(21.78531615561271*um_scale)
LiNbO3_gam3 = 0
LiNbO3_sig3 = 12.614

LiNbO3_susc_o = [mp.LorentzianSusceptibility(frequency=LiNbO3_frq1, gamma=LiNbO3_gam1, sigma_diag=LiNbO3_sig1*mp.Vector3(1,1,0)),
                 mp.LorentzianSusceptibility(frequency=LiNbO3_frq2, gamma=LiNbO3_gam2, sigma_diag=LiNbO3_sig2*mp.Vector3(1,1,0)),
                 mp.LorentzianSusceptibility(frequency=LiNbO3_frq3, gamma=LiNbO3_gam3, sigma_diag=LiNbO3_sig3*mp.Vector3(1,1,0))]

LiNbO3_frq1 = 1/(0.14307340773183533*um_scale)
LiNbO3_gam1 = 0
LiNbO3_sig1 = 2.9804
LiNbO3_frq2 = 1/(0.2580697580112788*um_scale)
LiNbO3_gam2 = 0
LiNbO3_sig2 = 0.5981
LiNbO3_frq3 = 1/(20.39803912144498*um_scale)
LiNbO3_gam3 = 0
LiNbO3_sig3 = 8.9543

LiNbO3_susc_e = [mp.LorentzianSusceptibility(frequency=LiNbO3_frq1, gamma=LiNbO3_gam1, sigma_diag=LiNbO3_sig1*mp.Vector3(0,0,1)),
                 mp.LorentzianSusceptibility(frequency=LiNbO3_frq2, gamma=LiNbO3_gam2, sigma_diag=LiNbO3_sig2*mp.Vector3(0,0,1)),
                 mp.LorentzianSusceptibility(frequency=LiNbO3_frq3, gamma=LiNbO3_gam3, sigma_diag=LiNbO3_sig3*mp.Vector3(0,0,1))]

LiNbO3 = mp.Medium(epsilon=1.0, E_susceptibilities=LiNbO3_susc_o+LiNbO3_susc_e, valid_freq_range=LiNbO3_range)
```

## Harminv

**Items to figure out:** 
1. How to use Harminv
    1. do you know what it does?

2. Extend to three dimensions to provide 2D confinement (this affects the waveguide dispersion).
    1. can you add the silica substrate? (this should also affect the waveguide dispersion)

## Useful Examples to look at:

1. mode-decomposition
    1. This calculates the reflection spectrum of a taper in a waveguide. It also shows you how to use **EigenModeSource**